In [3]:
import requests

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': '72c2183e4c034c15a0305818667390b7',
    'client_secret': '7dc6919a592f49519435f7dc3f8fc1b5',
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token= auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [10]:
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID from the URI
track_id = '6y0igZArWVi6Iz0rj35c1Y,7L7QYf7S6wg6pseN2C0d6K'

# actual GET request with proper header
r = requests.get(BASE_URL + 'audio-features?ids=' + track_id, headers=headers)

r=r.json()
r['audio_features'][1]

{'danceability': 0.661,
 'energy': 0.544,
 'key': 4,
 'loudness': -7.528,
 'mode': 1,
 'speechiness': 0.0782,
 'acousticness': 0.382,
 'instrumentalness': 0,
 'liveness': 0.164,
 'valence': 0.768,
 'tempo': 99.783,
 'type': 'audio_features',
 'id': '7L7QYf7S6wg6pseN2C0d6K',
 'uri': 'spotify:track:7L7QYf7S6wg6pseN2C0d6K',
 'track_href': 'https://api.spotify.com/v1/tracks/7L7QYf7S6wg6pseN2C0d6K',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7L7QYf7S6wg6pseN2C0d6K',
 'duration_ms': 142350,
 'time_signature': 4}

In [49]:
from time import sleep

def requestAPI(track_ids):
    AUTH_URL = 'https://accounts.spotify.com/api/token'

    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': '72c2183e4c034c15a0305818667390b7',
        'client_secret': '7dc6919a592f49519435f7dc3f8fc1b5',
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token= auth_response_data['access_token']

    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
    }

    BASE_URL = 'https://api.spotify.com/v1/'

    track_ids_str = ','.join(track_ids)
    # actual GET request with proper header
    r = requests.get(BASE_URL + 'audio-features?ids=' + track_ids_str, headers=headers)

    while r.status_code!=200:
        print('Status code ->',r.status_code)
        sleep(20)
        r = requests.get(BASE_URL + 'audio-features?ids=' + track_ids_str, headers=headers)

    r = r.json()
    result = []
    for track in r['audio_features']:
        result.append((track['energy'], track['danceability'], track['duration_ms'], track['instrumentalness'], track['loudness'], track['tempo'], track['valence']))
    return result

print(requestAPI(['0RmDknxxiA8fnKEIAbvVui', '1v0Yd9leQpwcbPxvUTQOGX']))


[(0.738, 0.698, 198984, 0, -5.338, 128.016, 0.3), (0.702, 0.586, 176260, 0, -7.265, 160.089, 0.359)]


In [29]:
from time import sleep
import redis 
import json
import requests

redis_cache= redis.StrictRedis(host='localhost')

def getAccessToken():
    AUTH_URL = 'https://accounts.spotify.com/api/token'

    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': '72c2183e4c034c15a0305818667390b7',
        'client_secret': '7dc6919a592f49519435f7dc3f8fc1b5',
    })
    
    auth_response_data = auth_response.json()
    return auth_response_data['access_token']


def getAuthHeader():

    header = redis_cache.get('header')
    if header == None:
        
        access_token = getAccessToken()
        header = {
            'Authorization': 'Bearer {token}'.format(token=access_token)
        }
        redis_cache.set('header',json.dumps(header),ex=3500)  #Spotify timeout at 3600
    else:
        header = json.loads(header)
    return header




def requestAPI(track_id):
    
    headers = getAuthHeader()
    
    track_id = '0RmDknxxiA8fnKEIAbvVui'
    BASE_URL = 'https://api.spotify.com/v1/'
    r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
    
    while r.status_code!=200:
        print('Status code ->',r.status_code)
        print(r.json())
        sleep(20)
        r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
    

    r = r.json()
    return (r['energy'],r['danceability'],r['duration_ms'], r['instrumentalness'],\
            r['loudness'],r['tempo'],r['valence'])


print(requestAPI('ciao'))

(0.738, 0.698, 198984, 0, -5.338, 128.016, 0.3)


In [5]:
import redis 

red= redis.StrictRedis(host='localhost')

red.set('pippo','solo',ex=1)

print(red.get('pippo').decode("utf-8"))

solo


In [24]:
redis_cache.delete('header')

1